In [6]:
import openai
import json
import os

# 设置 OpenAI API 密钥
openai.api_key = os.getenv("OPENAI_API_KEY")
# 检查API密钥是否正确设置
print(f"API Key exists: {'*****' if openai.api_key else 'No'}")
def analyze_financial_news(news_text):
    """
    调用 GPT-4o API 分析金融新闻文本，输出受影响国家的汇率评分及解释。
    
    要求：
    - 返回 JSON 格式对象数组，每个对象包含以下字段：
        - "country": 国家名称
        - "currency": 该国的货币代码
        - "impact_score": 在[-1,1]之间的评分（负值表示贬值风险，正值表示升值预期）
        - "explanation": 针对该国家的简要解释
        - "keywords": 新闻文本中涉及的经济关键词列表
    - 如果新闻中未检测到至少两个国家，则输出 {"error": "未检测到至少两个国家，无法进行汇率影响分析"}
    """
    
    prompt = f"""你是一个金融分析专家。请分析下面的金融新闻文本，并判断其中涉及的国家汇率变化的影响。请严格按照如下要求输出结果：
1. 输出 JSON 格式的对象数组，每个对象代表一个国家，包含以下字段：
   - "country": 国家名称
   - "currency": 该国的货币代码
   - "impact_score": 在[-1,1]之间的评分，要求是一个连续的值，其中负值表示汇率下降（不利影响），正值表示汇率上升（有利影响）
   - "explanation": 针对该国家给出的简要解释
   - "keywords": 新闻文本中涉及的经济指标关键词列表
2. 如果新闻中未检测到至少两个国家，请输出：{{"error": "未检测到至少两个国家，无法进行汇率影响分析"}}。
下面是金融新闻文本：
\"\"\"{news_text}\"\"\"
"""
    try:
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "你是一个金融分析专家。"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,  # 设置为 0 保证输出稳定
            max_tokens=1000
        )
        
        result_text = response.choices[0].message.content.strip()
        # 尝试解析 JSON 输出
        result = json.loads(result_text)
        return result
    except Exception as e:
        print("调用 GPT-4 API 时出错:", e)
        return None

# 示例调用
if __name__ == "__main__":
    news_text = (
        "在近期的市场波动中，美国公布的就业数据远超预期，刺激美元走强；"
        "而中国的出口数据却出现下滑，市场担心人民币可能面临贬值压力。"
        "欧洲则因能源价格波动，部分国家汇率有所波动。"
    )
    
    analysis_result = analyze_financial_news(news_text)
    if analysis_result:
        print(json.dumps(analysis_result, indent=2, ensure_ascii=False))
    else:
        print("分析失败，请检查 API 密钥或网络连接。")


API Key exists: *****
[
  {
    "country": "美国",
    "currency": "USD",
    "impact_score": 0.8,
    "explanation": "美国公布的就业数据远超预期，刺激美元走强。",
    "keywords": [
      "就业数据",
      "美元"
    ]
  },
  {
    "country": "中国",
    "currency": "CNY",
    "impact_score": -0.7,
    "explanation": "中国的出口数据出现下滑，市场担心人民币可能面临贬值压力。",
    "keywords": [
      "出口数据",
      "人民币"
    ]
  },
  {
    "country": "欧洲",
    "currency": "EUR",
    "impact_score": 0.2,
    "explanation": "欧洲因能源价格波动，部分国家汇率有所波动。",
    "keywords": [
      "能源价格",
      "汇率"
    ]
  }
]
